# Intro

This notebook will show you how to dock and score molecules using the asapdiscovery-docking module. 

This docking pipeline primarily focuses on the use-case for a structure-enabled drug discovery program, in which we have crystal structures of early molecules to use for *reference-based* docking. 

To this end, we have implemented an api that wraps the OpenEye POSIT docking algorithm, which through it's use of the HYBRID and SHAPEFIT algorithms, enables reference-based docking. 

## The scope of this guide

This guide will show you how to dock and score molecules. For the *extremely* necessary precursor step of data loading and prepping, please see [protein_and_ligand_prep](%protein_and_ligand_prep.ipynb)

# Data

For the 

# Protein and Ligand Preparation

# A few side notes

## Dask

We make heavy use of Dask throughout our code, which helps automate parallel processing and provides a nice dashboard for evaluating the progress of large scale docking efforts. Due to the way in which Dask automates error handling, this has occasionally led to situations where the behaviour of our code is different depending on whether you have enabled Dask. We have tried to stamp out any instances of this, but if you find another, please make an issue!

## Target-specific workflows

We have implemented our library code within the `asapdiscovery-workflows` module, which puts everything together in a command-line interface (cli). Unfortunately, as of version 0.4, these workflows only work if you are using the targets specified for ASAP. We plan on changing this for version 0.5 

To find out which targets can be passed to these workflows, you can use this:

In [11]:
from asapdiscovery.data.services.postera.manifold_data_validation import TargetTags

In [12]:
TargetTags.get_values()

['DENV-NS2B-NS3pro',
 'SARS-CoV-2-Mpro',
 'ZIKV-NS2B-NS3pro',
 'MERS-CoV-Mpro',
 'EV-D68-Capsid',
 'EV-D68-3Cpro',
 'SARS-CoV-2-N-protein',
 'SARS-CoV-2-Mac1',
 'EV-A71-3Cpro',
 'EV-A71-Capsid']